In [1]:
%load_ext snakeviz

In [1]:
from benchmark import Benchmark
from quantum import H_from_qubo, Qobj
from MaxCut import MaxCut

from qutip.parallel import parallel_map, serial_map

from qaoa_and_tdvp import QAOA, QAOAResult, qaoa_tdvp_rhs, tdvp_optimize_qaoa, scipy_optimize

import pickle

import numpy as np
from numpy.typing import NDArray
from typing import List, Tuple, Union, Optional, Callable, Any
import networkx as nx
import matplotlib.pyplot as plt
import plotly as py

import pandas as pd

In [2]:
n = 4
with open(f"./instances/n{n}_instances.p","rb") as f:
    instances = pickle.load(f)
delta = tuple(1 for _ in range(2 * n))
bench = Benchmark()

In [5]:
with open("./benchmarks/Euler_p1-5_n4.p.p", "rb") as f:
    euler_bench = pickle.load(f)

In [4]:
Delta = .1
min_p = 1
max_p = 4

In [24]:
# for this instance with p=4, Euler sovler did not succeed with grad_tol 1e-3 and Delta .1
qaoa_hard = QAOA(instances[0].qubo, p=4)
bench.run(qaoa_hard,delta_0=delta, tdvp_stepsize=1, p=4, tdvp_grad_tol=1e-3, tdvp_lineq_solver="RK45")


KeyboardInterrupt: 

In [ ]:
bench.save("n_4_first_instance_p_4_RK45")

In [3]:
results = list()
for num, i in enumerate(instances):
    print(f"instance {num}/{len(instances)}")
    instance_results = dict()
    for p in range(min_p, max_p+1):
        print(f"\t p = {p}/{max_p}")
        bench.run(
            qaoa=QAOA(i.qubo,p=p),
            delta_0=tuple(1 for _ in range(2 * p)),
            tdvp_stepsize=Delta,
            tdvp_grad_tol=1e-3,
            p=p,
            tdvp_lineq_solver="euler",
        )
        instance_results[p] = bench.results
    results.append((i, instance_results))
    bench.save("Euler_p1-5_n4.p")

instance 0/6
	 p = 1/4


KeyboardInterrupt: 

In [ ]:
n = 5
with open(f"./instances/n{n}_instances.p","rb") as f:
    instances = pickle.load(f)
delta = tuple(1 for _ in range(2 * n))
bench = Benchmark()

In [ ]:
Delta = .1
min_p =1
max_p = 4
results = list()
for num, i in enumerate(instances):
    print(f"instance {num}/{len(instances)}")
    instance_results = dict()
    for p in range(min_p, max_p+1):
        print(f"\t p = {p}/{max_p}")
        bench.run(
            qaoa=QAOA(i.qubo,p=p),
            delta_0=tuple(1 for _ in range(2 * p)),
            tdvp_stepsize=Delta,
            tdvp_grad_tol=1e-3,
            tdvp_lineq_solver="euler",
        )
        instance_results[p] = bench.results
    results.append((i, instance_results))
    bench.save("Euler_p1-5_n5.p")

In [ ]:
import os
os.system('shutdown -s -t 1')